## Image Captioning
We will discuss image captioning in different ways.
- Using spatial features(With only basic CNN and RNN)
- Using RNN and attention model
- Using Transformers

### Basic approach
From an image to varibly length of words. For this particular task we will use two types of neural network.
- Firstly, CNN to summarize the image into a single vector. which is the flatten vector.
- Secondly, a RNN. we will feed the sumarized vector into the RNN to gernerate caprion.
 
**Note**: 
- We need to specify the start and end token to the RNN to begin and terminate generating caption respectively.







![alt text](images/10.PNG)  ![alt text](images/11.PNG)

![alt text](images/sp1.PNG)

##### Gradient calculation:
gradient will be calculated through all the way to the last time step of RNN to the CNN weights.

##### Problem:
- model needs to everything it wants to say within one context vector "c" which we described in the attention section already.
This is a problem if we want to generate really long description. To solve this issue we address "Attention". Using "attention model" we initialize context vector at every time step instead of taking same context vector at each time step.

### Image Captioning with RNNs and Attention

We have learned attention method already. Same procedure will be applied here. Now input is an image instead of a sequence. Image will be passed through a CNN which outputs an feature matrix (H X W X D). Usually, initial decoder state $s_0$ is considered to be this output feature matrix. Now we will compute context vector in the same way we did for language translation. Decoder will use this initial decoder state and will compute pairwise allignment scores between this $s_0$ and each position in the grid of features. These values are scalar. Then we will use softmax to each of these values. This means that model is predicting probability distribution over all positions in the input image that it will choose to attend to when generating the first word of the output caption. Now the feature vector/initial decoder state will be used to compute the first context vector $c_1$ that wil use when generating the first token of the output sequence. Then we can repeat this process. While training, the model shift its attention around the image for every words that it generates in the caption.



#

![alt text](images/ic1.PNG) ![alt text](images/ic2.PNG)
![alt text](images/ic3.PNG) ![alt text](images/ic4.PNG)

#### Types of attention
There are two types of attention
- **Soft attention**: When we take weighted combination of all features from all image locations.
- **Hard attention**: We are forcing the model to look at exactly one position in the image at each time step.

So far we have discussed soft attention in the above. For hard attention there will be needed other mechanism which we wil discuss later.

### Some works related to attention model
"X, attend, and Y"

- Show, attend, and tell: Look at image, attend to image regions, produce question.
- Ask, attend, and answer: Read text of question, attend to image regions, produce answers.
- Listen, attend, and walk: Porcess text, attend to text regions, output navigation commands
- show, attend, and interact: Process image, attend to image regions, output robot control commands
- show, attend, and read: Process image, attend to image regions, output text

According to the above illustration, it can be said that "attention" method is used in several applications. Therfore we need to extract it more to generalize and apply in various problems.

### Generalyzing attention




we will have one query vector q, input vectors X. We will use a similarity function $f_{att}$ using these q and X to produce similarity vectors. Then we will run through a softmax function to give now a normalized probability distribution over each of the input vector x. Now we will output a single vector y that is weighted combination of the vectors X in the input. So now, first generalization **we want to change** the similarity function.

 Rather than running a neural network to compute these similarities we can compute all these similarities all at once with some kind of matrix multiplication and that will be much more efficient. Here the we will call this dot product as "scaled dot product" for computing these similarity scores. So now when computing the similarity score between the query vector q and one of our input vectors $x_i$ similarity score will be the q dot product with $x_i$ divide by square root $D_Q$ where $D_Q$ is the dimensionality of those two vectors. The intuition is that we are about to take those similarity scores and we are about to run them through a softmax. We know that if the elements of the softmax is really large then we will have a vanishing gradient problem. There are one of those $e_i$ in that attention weights e that is much much higher than others then we will end up with a very very highly peaked soft max distribution that will give us gradients very close to zero almost everywhere which might make learning challenging. Another problem is as we consider vectors of very very high dimension then their dot products are likely to also very high in magnitude. As a concrete example consider computing dot product of two vectors a and b both of dimension d and suppose these are constant vectors. Now the dot products of their magnitudes multiplied by the cosine of the angle between the vectors. Now if the vectors are constant then the magnitude of one of these vectors is square root of one of these vector. This means if we work with neural network that have very large dimensions then naturally we would expect these dot products of very high dimensional vectors to give rise to very high values. So to counteract, we are going to divide the dot product by the square root of the dimension to counteract this effect by which the dot product tends to scale as we scale up the dimension. This will give us nicely behaved gradients as we flow through softmax function.

![alt text](images/gn1.PNG)

The next generalization is that we want to allow multiple query vectors. So now we have set of query vectors and set of input vectors. We will compute probability distribution for each of the query over each of the input vectors. That means we need to compute the similarity between each query vector and each input vector. Because we use the scale dot product as our similarity function we can compute all of these similarity scores all simultaneously using a single matrix multiplication operation. We can apply softmax over these output attention scores where we take the softmax over only one of the dimensions. Now we will generate a set of output vectors one for each query vector. The output vector for query $q_i$ will be the combination of all of input vectors and they will be weighted by the distribution that we prdicted for the query vector. We will perfomr a single matrix multipication between these predicted attention weights and the input vectors X.


Next generalization is the way that we use the input vectors. Previously we are using the input vectors in two ways. First, we use it to compute attention weights by comparing each input vectors witht each query vector. Then we are using the input vectors again to produce the output. We can seperate the input vector into key vector and value vector. Now we will have learnable key matric $W_k$ and a larnable value matrix $W_v$. we will use these to transform the input vectors into two new set of vectors. One of the keys and one of the values. Now we will use these two different vectors for two different purposes in the computaion of the layer. So now, in order to computer query vector we compare each query vector with each key vector and then computing the output scores that the outputs are going to be weighted combnation of the value vectors that are predicted by the similarity scores. These gives the model flexibility to use the input data.

**Procedure**: Say we have input queries and input vectors. We will first generate key vectors from input vectors. Then we will apply similarity function using key and query vectors. These operation is actually a scale dot product between one of the key vectors and one of the query vectors. Thens apply softmax to these scores to normalize. Now these allignment scores gives us probability distribution over all the inputs x1,x2,..... each column represents x1,x2,...Now we need to compute the output. Again transform the input vectors into value vectors and perform weighted combination of the value vecotrs according to these computed allignment scores. Poduct is along horizontal way and the sum is along vertical way.

![alt text](images/q1.PNG)


#### **Self Attention Layer**: 
Another special case is self attention layer where we have only one set of vectors. Now we will compare each one of input vetors with another input vector. The way we do that we have another learnable weight matrix layer. So rather than taking the query vectors as input we are going to predict the query vectors by transforming the input vectors one at a time. Everything else is same as before. 

![alt text](images/sa1.PNG)

**Procedure:** We will transform input vectors into query vectors and key vectors. Then we will compute similarity scores using them. Then apply softmax as before. Then we convert the input vectors inot value vector again and perform again weighted similarity between the value vectors and sofmax scores to predict outputs. Again product is along horizontal and the sum is along vertical. **Self attention layer doesn't care about the order of its inputs**. We can recover the order by appending with some encoding of the position. One of the ways is to learn a lookup table and you add a learnable weight metrics to the network we are going to learn a vector for position one learn a vector position two and so on. This allows the model to distinguish which part of the model is the begining and which part of the model is the end. This is sometimes called positional 
encoding

![alt text](images/pe.PNG)


#### Masked self attention model

Another variant of self attention layer is Maked self attention layer. In this type the model is to use information from the past. In tasks like language modeling we want to ask the network to predict token given all of the previous tokens. Using this default attention layer the model is allowed to use on every vector when producing the output vector. So, this won't work for the language model. But we can do this by adding some structures to this attention matrix. we can manually intervene the predicted matrix "e". we can just put minus infinity to the positions we want to force the model not to pay attention to things. We want first output to only depend on first input. So we can just put minus infinity to those parts of the matrix. The softmax will put zero for these positions.

![alt text](images/masked.PNG)

#### Multi-head self attentin model

Another type is multiheaded self attention. In this type we will choose numbers of heads h and will run h self attention layers independently parllel. So, then given our set of input vectors x. if our vector x has a dimension d then we will split each of our input vectors into h chunks of equal size and then feed these chunks into separate parllel of self attention layers. Then each self attention layer produce some set of outputs one output for each inputs then we will concatenate those outputs to get the final output from this multiheaded self attention layer. input dimesion of the query x is the input dimension of data, the output dimension of y is the dimension we want to predict.



![alt text](images/multihead.PNG)

#### CNN with Self-Attention layer

![alt text](images/cnn-sa1.PNG)



From an input image we extract features of size C X H X W. Now we will use three different one by one convolutions to convert our grid of features into a grid of query, a grid of keys, and a grid of values. These will be converted by three separate one-by-one convolutions with their own weights and biases that are learned. Now, we will compute inner products between the queries and the keys that will give us attention and the we will apply softmax and that will give us for every position in the input image how much does it want to attend to every other position in the input image. Then this will generate this very very large matrix of attention weights of size (HXW)X(HXW). Now we will use this attention weights to have this weighted linear combination of the value vectors and will end up producing one a value vector for each position in the input. This means every input vector from the feature vector we are producing new grid of feature vectors. Now every position in the output grid depends on every position in the input grid. Sometimes people add another 1X1 convolution after the end of this attention layer. Moreover it's common to add a residual connection around this entire self attention operation

#### Three ways of processing sequence
- RNN: problem is not parallellizable. need to compute hidden state sequentially.
- 1D Convolution: not very good for long sequence
- Self Attention: very memory intensive

## Transformers
If we want to build neural network that process sequences we can do it using only self attention layer. The new primitive block type called the Transformer block. We will apply self attention to a set of inputs. After the self attention layer each output of the self attention layer depend on each input, that gives us our interaction between each element between all elements of the sequence or set. After self attention we will add a residual net around self attention that will improve the gradient flow through the model. After residual connection we will add Layer normalization. Layer normalization is applied to each of the output vectors from self attention layer independently, which means no interaction between these vectors. After layer normalization we will apply multi-layer perceptron that is why we call it a fully connected layer. The output of layer normalization is set of vectors and we will run it through a fully connected layer. Then we will another residual net around the MLP. Then we will add another layer normalization. Then we will put all these into a single block which we will call a transformer block. So the input of trnasformer block is a set of vectors x the output is also a number of vectors y. The number of input vectors and output vectors are same but might have different dimensionality. And a transformer model is just a sequence of transformer blocks
**Hyperparameters**:
- overall depth: number of blocks